In [1]:
from MOT import MOT
import numpy as np
import matplotlib.pyplot as plt
import os
from BfTableGenerator import RansacCollector
from BfTableGenerator import TDmapLoader
from tqdm import tqdm
# from open3d import JVisualizer 
from Utils import *
from DDBSCAN import Raster_DBSCAN
import open3d as op3
# op3.visualization.webrtc_server.enable_webrtc()
from open3d.web_visualizer import draw
from VisulizerTools import *
from numpy.lib.stride_tricks import sliding_window_view
from scipy.stats import multivariate_normal



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [ ]:
draw()

In [144]:
def get_affinity_mat_jpd_TR(state,state_,P_,mea):
    State_affinity = np.zeros((state_.shape[1],state_.shape[0],mea.shape[0]))
    for i,s_ in enumerate(state_):
         # includes the pred states for two reprs 
         # s_: 2 x 6 x 1
        state_cur = state[i].copy().reshape(2,-1)[:,:2]
        state_pred = s_.copy().reshape(2,-1)[:,:2]
        
         # cov_tr : 2 x 6 x 6 
        cov_tr = P_[i][:,:2,:2]
        var_tr = [multivariate_normal(mean=state_pred[k], cov=cov_tr[k]) for k in range(state_cur.shape[0])]
        for j,m in enumerate(mea):
            mea_next = m.copy().reshape(2,-1)
            for k in range(s_.shape[0]):
                dis_error = np.sqrt(np.sum((state_pred[k] - mea_next[k])**2))
                if dis_error < 5:
                    jp = var_tr[k].pdf(mea_next[k])
                    State_affinity[k,i,j] = jp
    
    return np.max(State_affinity,axis = 0)

In [4]:
def get_representative_point(rows,cols,Td_map): 
    td_freq_map = Td_map
    longitudes = theta[rows]*np.pi / 180
    latitudes = azimuths[cols] * np.pi / 180 
    hypotenuses = td_freq_map[rows,cols] * np.cos(longitudes)
    X = hypotenuses * np.sin(latitudes)
    Y = hypotenuses * np.cos(latitudes)
    Z = td_freq_map[rows,cols] * np.sin(longitudes)
    
    return np.array([X,Y]).reshape(-1,2,1) # n_repr x xy_dim x 1 

In [2]:
# os.chdir(r'/Users/czhui960/Documents/Lidar/RawLidarData/FrameSamplingTest')
# thred_map = np.load(r'Output File/thred_map_1200.npy')
collector = RansacCollector(pcap_path=r'../RawLidarData/Veteran/Veteran.pcap',output_file_path='../MergingResult/',update_frame_num=2000)
collector.gen_tdmap()
collector.gen_thredmap(d = 1,thred_s = 0.2,N = 20,delta_thred = 1e-3,step = 0.1)

Loading pcap...


100%|███████████████████████████████████████| 2000/2000 [01:20<00:00, 24.89it/s]


Generating Threshold Map


In [3]:
db = Raster_DBSCAN(window_size=(5,13),eps = 1.7, min_samples= 17,Td_map_szie=collector.thred_map.shape)   

In [22]:
for f in tqdm(range(2000)):
    Td_map = collector.aggregated_map[f]
    Foreground_map = (Td_map < collector.thred_map)&(Td_map != 0)
    Labeling_map = db.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
    Background_map = (Td_map >= collector.thred_map)&(Td_map != 0)
    for r in range(Labeling_map.shape[0]):
        if (Labeling_map[r,0] != -1) & (Labeling_map[r,-1] != -1) & (Labeling_map[r,-1] != Labeling_map[r,0]):
            print(f)
            break

 74%|███████▍  | 1487/2000 [00:59<00:32, 15.65it/s]

1484
1485


100%|██████████| 2000/2000 [01:27<00:00, 22.92it/s]


In [4]:
Td_map = collector.aggregated_map[0]
Foreground_map = (Td_map < collector.thred_map)&(Td_map != 0)
Labeling_map = db.fit_predict(Td_map= Td_map,Foreground_map=Foreground_map)
Background_map = (Td_map >= collector.thred_map)&(Td_map != 0)

In [5]:
def get_pcd_colored(Td_map,Labeling_map,Foreground_map):
    
    Xs = []
    Ys = []
    Zs = []
    Labels = []
    for i in range(Td_map.shape[0]):
        longitudes = theta[i]*np.pi / 180
        latitudes = azimuths * np.pi / 180 
        hypotenuses = Td_map[i] * np.cos(longitudes)
        X = hypotenuses * np.sin(latitudes)
        Y = hypotenuses * np.cos(latitudes)
        Z = Td_map[i] * np.sin(longitudes)
        Valid_ind = (Td_map[i] != 0)&(Foreground_map[i])
        Xs.append(X[Valid_ind])
        Ys.append(Y[Valid_ind])
        Zs.append(Z[Valid_ind])
        Labels.append(Labeling_map[i][Valid_ind])

    Xs = np.concatenate(Xs)
    Ys = np.concatenate(Ys)
    Zs = np.concatenate(Zs)
    Labels = np.concatenate(Labels)
    XYZ = np.concatenate([Xs.reshape(-1,1),Ys.reshape(-1,1),Zs.reshape(-1,1)],axis = 1)
    Colors = np.full((len(Labels),3),np.array([[153,153,153]])/256)
    unique_label = np.unique(Labels)
    for l in unique_label[1:]:
        Colors[Labels == l] = color_map[l%len(color_map)]

    pcd = op3.geometry.PointCloud()
    pcd.points = op3.utility.Vector3dVector(XYZ)
    pcd.colors = op3.utility.Vector3dVector(Colors)
    return pcd    

In [6]:
pcd = get_pcd_colored(Td_map,Labeling_map,Foreground_map)

In [ ]:
draw(pcd)